# Ensemble Techniques Project

Program that will take in a dataset's location and target column, go through 4 ML models and store the best performing model based on your performance priority.

The 4 ML models are;
1. Logistic Regression
2. SVM
3. Random Forest
4. CatBoost

It runs through the default model and a hypertuned model for the above mentioned and chooses the best based on your priority.

Randomized search has been employed with wide ranges for fast, yet practical results.

In [11]:
# Importing basic data science libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import random

#Importing ML metric evaluation.
from sklearn.metrics import classification_report, confusion_matrix

#Importing model split
from sklearn.model_selection import train_test_split

#Importing model selection libraries.
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Importing ML algorithms.
#Part A
import xgboost as xgb

#Part B
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

#Importing miscelleaneous libraries.
from pandas.api.types import is_numeric_dtype, is_string_dtype
from sklearn.preprocessing import StandardScaler

#Importing imputation libraries.
from imblearn.over_sampling import ADASYN



## Function definitions:

### Loading the dataset.

In [12]:
def load_dataset(p_csv_file): # Function to load csv file into dataset.
  return pd.read_csv("D:/Great Learning AIML/Ensemble Techniques/Ensemble Techniques Project/"+p_csv_file)

### Getting the characteristics of dataframe.

In [13]:
def get_attr_of_df(p_df): # Function to get characteristics of dataframe.
  f_rows, f_columns = p_df.shape # To get number of rows and columns in dataframe.
  f_df_describe = p_df.describe(include='all') # To get statistic metrics of dataframe.
  f_df_dtypes = p_df.dtypes # To get datatype of each column.

  return ( f_rows, f_columns, f_df_describe, f_df_dtypes )

### Checking for NaN values.

In [14]:
def check_for_NaN(p_df):
  f_null_counts = p_df.isna().sum() # Check for Null/NaN values.

  for values in f_null_counts:
    if values != 0:
      return True # Found NaN values

  return False # Didn't find NaN values.

### Getting columns that contain NaN values.

In [15]:
def get_NaN_columns(p_df):
  f_null_counts = p_df.isna().sum() # Check for Null/NaN values.

  f_null_columns = list()
  for row, value in zip(f_null_counts.index.to_list(), f_null_counts):
    if value != 0:
      f_null_columns.append(row) # Store column names that have NaN/Null values.

  return f_null_columns

### Dealing with NaN values.

In [16]:
def deal_with_NaN(p_null_columns, p_df, p_df_describe): # Function to deal with NaN/Null values.
  for i in p_null_columns:
    if is_numeric_dtype(p_df[i]): # Checking if datatype is numeric
      p_df[i].fillna(p_df_describe.loc[("50%", i)], inplace=True) # If numeric use median to fill NaN values.
    elif is_string_dtype(p_df[i]): # Checking if datatype is string/object
      p_df[i].fillna(p_df_describe.loc[("top", i)], inplace=True) # Id string/object use mode to fill NaN values.


### Checking if output column is balanced.

In [17]:
def check_if_balanced(df):  
  list_imbalanced = list()

  temp_arr = df[df.columns.to_list()[-1]].unique() # Getting unique values present in target column.
  balancing_temp = df[df.columns.to_list()[-1]].value_counts() # Getting sum of each unique value in target column.

  for i, j in zip(temp_arr, balancing_temp): # Computing percentage of each unique value.
    list_imbalanced.append( ( balancing_temp[i]/balancing_temp.sum() ) * 100 )

  list_diff_imbalanced = list()
  for i in range(0, ( len(list_imbalanced) - 1)): # Computing absolute difference between each unique value.
    list_diff_imbalanced.append( abs(list_imbalanced[i] - list_imbalanced[i+1]) )

  for i in list_diff_imbalanced: # Checking if difference is above threshold or not.
    if i > 5:
      return False # Output is imbalanced.
  
  return True # Output is balanced.

### Converting categorical data to numeric format.

In [18]:
def convert_categorical(p_df):
  # Label encoding.
  for i, j in zip(p_df.columns, p_df.dtypes):
    temp = p_df[i].value_counts()
    temp_rows, = temp.shape
    if j == "object":
      if temp_rows == 2:
        print(f"\x1b[31mColumn:\x1b[0m \x1b[34m{i}\x1b[0m")
        print(f"\x1b[36mUnique values:\x1b[0m \x1b[32m{p_df[i].unique()}\x1b[0m")
        abc_0 = p_df[i].unique()[0] + ": "
        abc_1 = p_df[i].unique()[1] + ": "  
        bin_1 = 1
        bin_0 = 0
        print("Assign labels to features:")
        bin_0 = int(input(abc_0))
        bin_1 = int(input(abc_1))
        p_df[i] = p_df[i].map({p_df[i].unique()[0]: bin_0, p_df[i].unique()[1]: bin_1})
      
      elif temp_rows > 2 and temp_rows < 5:
        print(f"\x1b[31mColumn:\x1b[0m \x1b[34m{i}\x1b[0m")
        print(f"\x1b[36mUnique values:\x1b[0m \x1b[32m{p_df[i].unique()}\x1b[0m")
        print(p_df[i].unique())
        decision_1 = input("Do you want to manually encode this, Y/N?")
        if decision_1 == "N":
          continue
        else:
          bin_temp = dict()
          print(f"Assign labels features:")
          for a in p_df[i].unique():
            temp_2 = int(input(a+": "))
            bin_temp[a] = temp_2
          p_df[i] = p_df[i].map(bin_temp)
          bin_temp.clear()
  # One-hot encoding of remaining categorical columns.
  p_df.drop(["customerID"], axis=1, inplace = True)
  p_df = pd.get_dummies(p_df)
  return p_df

### Balancing data if it isn't using ADASYN.

In [19]:
def ADASYN_balancing(df):
  adasyn = ADASYN(random_state=42)
  target_col_name = input("Enter target column name: ")
  X = df.drop([ target_col_name ], axis=1) # Dropping last row which is the target column
  Y = df[target_col_name]
  X_ada, Y_ada = adasyn.fit_resample(X, Y) #Fitting and resampling the data.
  df_balanced = pd.concat([X_ada, Y_ada], join='outer', axis=1)
  '''
  df_balanced.drop(['index'], axis=1, inplace=True)
  df_balanced.reset_index(inplace=True)
  '''

  return df_balanced

## Preprocessing function.

In [20]:
def preprocess_data():
  print("Store the csv in the following folder;\nD:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project")
  print("Type in csv file name stored in above mentioned folder: ")
  csv_file = input()
  #csv_file = "Part_B.csv"
  #csv_file = "train.csv"

  df = load_dataset(csv_file) # Loading csv file into dataframe.
  target_col = input("Enter target column name from above list: ")

  rows, columns, df_describe, df_dtypes = get_attr_of_df(df) # Getting characteristics of dataframe.
  
  if check_for_NaN(df): # Check if any/all columns contain NaN values.
    print("Dataframe contains NaN values.")
  else:
    print("Dataframe doesn't contain any NaN values.")

  null_columns = get_NaN_columns(df) # Get any/all columns contain NaN values.

  #print(f"Before\n{df.isna().sum()}")

  if len(null_columns) != 0: # Check if any column has NaN values, if so, call function to deal with it.
    deal_with_NaN(null_columns, df, df_describe)

  #print(f"After\n{df.isna().sum()}")

  if check_for_NaN(df): # Check if any/all columns contain NaN values post Imputation.
    print("Dataframe contains NaN values.")
  else:
    print("Dataframe doesn't contain any NaN values.")

  df_conv = convert_categorical(df.copy()) # Run this in the end.#################################################################################

  if check_if_balanced(df_conv.copy()): # Check if target column is balanced.
    print("Output is balanced and we can proceed")
  else:
    print("Output is not balanced.")  
    df_preprocessed = ADASYN_balancing(df_conv.copy()) # Balancing the data using ADASYN method
  
  if check_if_balanced(df_preprocessed.copy()): # Verify if balanced.
    print("Output is balanced and we can proceed")
    
  return (df_preprocessed, target_col)
    #print(temp_df)

## Machine Learning Algorithms

### Logistic Regression

#### Base Model

In [21]:
def LogReg_base_model(df):
  logReg_base = LogisticRegression() # Creating base model
  #Let's split the data into 80-20 ratio
  X = df.drop(["Churn"], axis=1)
  Y = df["Churn"]

  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1, stratify = Y)

  logReg_base.fit(X_train, Y_train) # Training model on train data

  # Making predictions
  y_train_pred_logReg_base = logReg_base.predict(X_train)
  y_test_pred_logReg_base = logReg_base.predict(X_test)

  # Printing and storing performance metrics of both training and testing data.
  cr_train_logReg_base = classification_report(Y_train, y_train_pred_logReg_base, output_dict=True)
  cr_test_logReg_base = classification_report(Y_test, y_test_pred_logReg_base, output_dict=True)
  print("\x1b[1;34mLogistic Regression Base model Training set\x1b[0m")
  print(classification_report(Y_train, y_train_pred_logReg_base))
  print("\x1b[1;34mLogistic Regression Base model Testing set\x1b[0m")
  print(classification_report(Y_test, y_test_pred_logReg_base))



  return (cr_train_logReg_base, cr_test_logReg_base, logReg_base)

#### Hypertuned Model

In [22]:
def LogReg_hyper_model(df):
  '''
  Okay so to improve peformance, we can
  1. Standardize/Normalize the data
  2. Balance the data(which is already done)
  3. Hypertune the parameters using GridSearchCV
  '''
  logReg_hyper = LogisticRegression()

  #Standardizing the data.
  logReg_scalar = StandardScaler()
  X = df.drop(["Churn"], axis=1)
  Y = df["Churn"]

  X_scaled = logReg_scalar.fit_transform(X)

  X_train_scaled, X_test_scaled, Y_train_scaled, Y_test_scaled = train_test_split(X_scaled, Y, random_state=2, stratify=Y)

  #Hypertuning the model parameters.

  logReg_params = {
                  'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                   'C': [10, 1.0, 0.1]
                  }

  gsearch_logReg = GridSearchCV(estimator = logReg_hyper,
                                param_grid = logReg_params,
                                n_jobs = -1)
  
  gsearch_logReg.fit(X_train_scaled, Y_train_scaled)
  logReg_best_params_dict = gsearch_logReg.best_params_
  print(logReg_best_params_dict)
  logReg_hyper = LogisticRegression(solver = logReg_best_params_dict["solver"],
                                    C = logReg_best_params_dict["C"],
                                    )
  
  logReg_hyper.fit(X_train_scaled, Y_train_scaled)

  y_train_pred_logReg_hyper = logReg_hyper.predict(X_train_scaled)
  y_test_pred_logReg_hyper = logReg_hyper.predict(X_test_scaled)

  cr_train_logReg_hyper = classification_report(Y_train_scaled, y_train_pred_logReg_hyper, output_dict=True)
  cr_test_logReg_hyper = classification_report(Y_test_scaled, y_test_pred_logReg_hyper, output_dict=True)
  print("\x1b[1;34mLogistic Regression Hypertuned model Training set\x1b[0m")
  print(classification_report(Y_train_scaled, y_train_pred_logReg_hyper))
  print("\x1b[1;34mLogistic Regression Hypertuned model Testing set\x1b[0m")
  print(classification_report(Y_test_scaled, y_test_pred_logReg_hyper))

  return (cr_train_logReg_hyper, cr_test_logReg_hyper, logReg_hyper)

### SVM

#### Base Model

In [23]:
def SVM_base_model(df):
  svm_base = SVC()

  X = df.drop(["Churn"], axis=1)
  Y = df["Churn"]

  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=3, stratify = Y)

  svm_base.fit(X_train, Y_train)

  y_train_pred_svm_base = svm_base.predict(X_train)
  y_test_pred_svm_base = svm_base.predict(X_test)

  cr_train_svm_base = classification_report(Y_train, y_train_pred_svm_base, output_dict=True)
  cr_test_svm_base = classification_report(Y_test, y_test_pred_svm_base, output_dict=True)
  print("\x1b[1;31mSVM Base model Training set\x1b[0m")
  print(classification_report(Y_train, y_train_pred_svm_base))
  print("\x1b[1;31mSVM Base model Testing set\x1b[0m")
  print(classification_report(Y_test, y_test_pred_svm_base))

  return (cr_train_svm_base, cr_test_svm_base, svm_base)

#### Hypertuned Model

In [24]:
def SVM_hyper_model(df):
  '''
  Okay so to improve peformance, we can
  1. Standardize/Normalize the data
  2. Hypertune the parameters using GridSearchCV
  '''

  svm_hyper = SVC()

  X = df.drop(["Churn"], axis=1)
  Y = df["Churn"]

  svm_scalar = StandardScaler()

  X_scaled = svm_scalar.fit_transform(X)

  X_train_scaled, X_test_scaled, Y_train_scaled, Y_test_scaled = train_test_split(X_scaled, Y, random_state=4, stratify=Y)

  svm_params = {'C': [0.1, 1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

  gsearch_svm = GridSearchCV(estimator = svm_hyper,
                             param_grid = svm_params,
                             refit=True)
  
  gsearch_svm.fit(X_train_scaled, Y_train_scaled)
  svm_best_params_dict = gsearch_svm.best_params_
  print(svm_best_params_dict)
  svm_hyper = SVC(
                  gamma = svm_best_params_dict["gamma"],
                  C = svm_best_params_dict["C"],
                  kernel = svm_best_params_dict["kernel"]
                  )
  
  svm_hyper.fit(X_train_scaled, Y_train_scaled)

  y_train_pred_svm_hyper = svm_hyper.predict(X_train_scaled)
  y_test_pred_svm_hyper = svm_hyper.predict(X_test_scaled)

  cr_train_svm_hyper = classification_report(Y_train_scaled, y_train_pred_svm_hyper, output_dict=True)
  cr_test_svm_hyper = classification_report(Y_test_scaled, y_test_pred_svm_hyper, output_dict=True)
  print("\x1b[1;31mSVM Hypertuned model Training set\x1b[0m")
  print(classification_report(Y_train_scaled, y_train_pred_svm_hyper))
  print("\x1b[1;31mSVM Hypertuned model Testing set\x1b[0m")
  print(classification_report(Y_test_scaled, y_test_pred_svm_hyper))

  return (cr_train_svm_hyper, cr_test_svm_hyper, svm_hyper)

### Random Forest

#### Base Model

In [25]:
def RF_base_model(df):
  RF_base = RandomForestClassifier()

  X = df.drop(["Churn"], axis=1)
  Y = df["Churn"]

  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=5, stratify = Y)

  RF_base.fit(X_train, Y_train)

  y_train_pred_RF_base = RF_base.predict(X_train)
  y_test_pred_RF_base = RF_base.predict(X_test)

  cr_train_RF_base = classification_report(Y_train, y_train_pred_RF_base, output_dict=True)
  cr_test_RF_base = classification_report(Y_test, y_test_pred_RF_base, output_dict=True)
  print("\x1b[1;32mRandom Forest Base model Training set\x1b[0m")
  print(classification_report(Y_train, y_train_pred_RF_base))
  print("\x1b[1;32mRandom Forest Base model Testing set\x1b[0m")
  print(classification_report(Y_test, y_test_pred_RF_base))

  return (cr_train_RF_base, cr_test_RF_base, RF_base)

#### Hypertuned Model

In [26]:
def RF_hyper_model(df):
  '''
  Okay so to improve peformance, we can
  1. Hypertune the parameters using GridSearchCV
  '''

  RF_hyper = RandomForestClassifier()

  X = df.drop(["Churn"], axis=1)
  Y = df["Churn"]

  RF_scalar = StandardScaler()

  X_scaled = RF_scalar.fit_transform(X)

  X_train_scaled, X_test_scaled, Y_train_scaled, Y_test_scaled = train_test_split(X_scaled, Y, random_state=6, stratify=Y)

  RF_params = {
               'n_estimators': [int(x) for x in np.arange(100, 501, 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.arange(1, 50, 1)],
               'min_samples_split': [int(x) for x in np.arange(2, 10, 1)],
               'min_samples_leaf': [int(x) for x in np.arange(2, 10, 1)],
               'bootstrap': [True, False]
               }

  gsearch_RF = RandomizedSearchCV(estimator = RF_hyper,
                             param_distributions = RF_params,
                             refit=True)
  
  gsearch_RF.fit(X_train_scaled, Y_train_scaled)
  RF_best_params_dict = gsearch_RF.best_params_
  print(RF_best_params_dict)
  RF_hyper = RandomForestClassifier(
                  n_estimators = RF_best_params_dict["n_estimators"],
                  max_features = RF_best_params_dict["max_features"],
                  max_depth = RF_best_params_dict["max_depth"],
                  min_samples_split = RF_best_params_dict["min_samples_split"],
                  min_samples_leaf = RF_best_params_dict["min_samples_leaf"],
                  bootstrap = RF_best_params_dict["bootstrap"]
                  )
  
  RF_hyper.fit(X_train_scaled, Y_train_scaled)

  y_train_pred_RF_hyper = RF_hyper.predict(X_train_scaled)
  y_test_pred_RF_hyper = RF_hyper.predict(X_test_scaled)

  cr_train_RF_hyper = classification_report(Y_train_scaled, y_train_pred_RF_hyper, output_dict=True)
  cr_test_RF_hyper = classification_report(Y_test_scaled, y_test_pred_RF_hyper, output_dict=True)
  print("\x1b[1;32mRandom Forest Hypertuned model Training set\x1b[0m")
  print(classification_report(Y_train_scaled, y_train_pred_RF_hyper))
  print("\x1b[1;32mRandom Forest Hypertuned model Testing set\x1b[0m")
  print(classification_report(Y_test_scaled, y_test_pred_RF_hyper))

  return (cr_train_RF_hyper, cr_test_RF_hyper, RF_hyper)

### CatBoost

#### Base Model

In [27]:
def CAT_base_model(df):
  CAT_base = CatBoostClassifier(logging_level='Silent', metric_period=10)

  X = df.drop(["Churn"], axis=1)
  Y = df["Churn"]

  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=7, stratify = Y)

  CAT_base.fit(X_train, Y_train)

  y_train_pred_CAT_base = CAT_base.predict(X_train)
  y_test_pred_CAT_base = CAT_base.predict(X_test)

  cr_train_CAT_base = classification_report(Y_train, y_train_pred_CAT_base, output_dict = True)
  cr_test_CAT_base = classification_report(Y_test, y_test_pred_CAT_base, output_dict = True)
  print("\x1b[1;35mCatBoost Base model Training set\x1b[0m")
  print(classification_report(Y_train, y_train_pred_CAT_base))
  print("\x1b[1;35mCatBoost Base model Testing set\x1b[0m")
  print(classification_report(Y_test, y_test_pred_CAT_base))

  return (cr_train_CAT_base, cr_test_CAT_base, CAT_base)

#### Hypertuned Model

In [28]:
def CAT_hyper_model(df):
  '''
  Okay so to improve peformance, we can
  1. Hypertune the parameters using GridSearchCV
  '''

  CAT_hyper = CatBoostClassifier()

  X = df.drop(["Churn"], axis=1)
  Y = df["Churn"]

  CAT_scalar = StandardScaler()

  X_scaled = CAT_scalar.fit_transform(X)

  X_train_scaled, X_test_scaled, Y_train_scaled, Y_test_scaled = train_test_split(X_scaled, Y, random_state=8, stratify=Y)

  CAT_params = {
                 'iterations': [int(x) for x in np.arange(10, 1001, 10)],
                 'depth': [int(x) for x in np.arange(1, 11)],
                 'learning_rate': [float(x) for x in np.arange(0.01, 1.0, 0.01)],
                 'random_strength': [float(x) for x in np.arange(0.01, 11, 0.01)]
                 }

  rand_search_CAT = RandomizedSearchCV(estimator = CAT_hyper,
                             param_distributions = CAT_params,
                             refit=True)
  
  rand_search_CAT.fit(X_train_scaled, Y_train_scaled)
  CAT_best_params_dict = rand_search_CAT.best_params_
  print(CAT_best_params_dict)
  CAT_hyper = CatBoostClassifier(
                  iterations = CAT_best_params_dict["iterations"],
                  depth = CAT_best_params_dict["depth"],
                  learning_rate = CAT_best_params_dict["learning_rate"],
                  random_strength = CAT_best_params_dict["random_strength"],
                  metric_period = 1,
                  silent=True
                  )
  
  CAT_hyper.fit(X_train_scaled, Y_train_scaled)

  y_train_pred_CAT_hyper = CAT_hyper.predict(X_train_scaled)
  y_test_pred_CAT_hyper = CAT_hyper.predict(X_test_scaled)

  cr_train_CAT_hyper = classification_report(Y_train_scaled, y_train_pred_CAT_hyper, output_dict=True)
  cr_test_CAT_hyper = classification_report(Y_test_scaled, y_test_pred_CAT_hyper, output_dict=True)
  print("\x1b[1;35mCatBoost Hypertuned model Training set\x1b[0m")
  print(classification_report(Y_train_scaled, y_train_pred_CAT_hyper))
  print("\x1b[1;35mCatBoost Hypertuned model Testing set\x1b[0m")
  print(classification_report(Y_test_scaled, y_test_pred_CAT_hyper))

  return (cr_train_CAT_hyper, cr_test_CAT_hyper, CAT_hyper)

### Pickle

#### Accuracy 1 model Pickle

In [29]:
def accuracy_1_model_pickle(accuracy_dict, model_list):

  temp_base_value = max(accuracy_dict['1']['base'].values()) # get max key, value pair of base model
  temp_base_key = list( accuracy_dict['1']['base'].keys() )[ list( accuracy_dict['1']['base'].values() ).index(temp_base_value) ]

  temp_hyper_value = max(accuracy_dict['1']['hyper'].values()) # get max key, value pair of hyper model
  temp_hyper_key = list( accuracy_dict['1']['hyper'].keys() )[ list( accuracy_dict['1']['hyper'].values() ).index(temp_hyper_value) ]

  abs_max = max(temp_base_value, temp_hyper_value) # Find which has best, base or hyper model
  
  if abs_max == temp_base_value: # If max value is in BASE model.
    if temp_base_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[0], open('D:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project\logReg_base_model_accuracy_1.pkl', 'wb'))
    elif temp_base_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[1], open('D:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project\SVM_base_model_accuracy_1.pkl', 'wb'))
    elif temp_base_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[2], open('D:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project\RF_base_model_accuracy_1.pkl', 'wb'))
    elif temp_base_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[3], open('D:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project\CAT_base_model_accuracy_1.pkl', 'wb'))
  
  elif abs_max == temp_hyper_value: # If max value is in HYPER model.
    if temp_hyper_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[4], open('D:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project\logReg_hyper_model_accuracy_1.pkl', 'wb'))
    elif temp_hyper_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[5], open('D:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project\SVM_hyper_model_accuracy_1.pkl', 'wb'))
    elif temp_hyper_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[6], open('D:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project\RF_hyper_model_accuracy_1.pkl', 'wb'))
    elif temp_hyper_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[7], open('D:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project\CAT_hyper_model_accuracy_1.pkl', 'wb'))

#### Precision 1 model pickle

In [30]:
def precision_1_model_pickle(precision_dict, model_list):

  temp_base_value = max(precision_dict['1']['base'].values()) # get max key, value pair of base model
  temp_base_key = list( precision_dict['1']['base'].keys() )[ list( precision_dict['1']['base'].values() ).index(temp_base_value) ]

  temp_hyper_value = max(precision_dict['1']['hyper'].values()) # get max key, value pair of hyper model
  temp_hyper_key = list( precision_dict['1']['hyper'].keys() )[ list( precision_dict['1']['hyper'].values() ).index(temp_hyper_value) ]

  abs_max = max(temp_base_value, temp_hyper_value) # Find which has best, base or hyper model
  
  if abs_max == temp_base_value: # If max value is in BASE model.
    if temp_base_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[0], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_base_model_precision_1.pkl', 'wb'))
    elif temp_base_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[1], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_base_model_precision_1.pkl', 'wb'))
    elif temp_base_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[2], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_base_model_precision_1.pkl', 'wb'))
    elif temp_base_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[3], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_base_model_precision_1.pkl', 'wb'))
  
  elif abs_max == temp_hyper_value: # If max value is in HYPER model.
    if temp_hyper_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[4], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_hyper_model_precision_1.pkl', 'wb'))
    elif temp_hyper_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[5], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_hyper_model_precision_1.pkl', 'wb'))
    elif temp_hyper_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[6], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_hyper_model_precision_1.pkl', 'wb'))
    elif temp_hyper_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[7], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_hyper_model_precision_1.pkl', 'wb'))

#### Recall 1 model pickle

In [31]:
def recall_1_model_pickle(recall_dict, model_list):
  temp_base_value = max(recall_dict['1']['base'].values()) # get max key, value pair of base model
  temp_base_key = list( recall_dict['1']['base'].keys() )[ list( recall_dict['1']['base'].values() ).index(temp_base_value) ]

  temp_hyper_value = max(recall_dict['1']['hyper'].values()) # get max key, value pair of hyper model
  temp_hyper_key = list( recall_dict['1']['hyper'].keys() )[ list( recall_dict['1']['hyper'].values() ).index(temp_hyper_value) ]

  abs_max = max(temp_base_value, temp_hyper_value) # Find which has best, base or hyper model
  
  if abs_max == temp_base_value: # If max value is in BASE model.
    if temp_base_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[0], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_base_model_recall_1.pkl', 'wb'))
    elif temp_base_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[1], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_base_model_recall_1.pkl', 'wb'))
    elif temp_base_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[2], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_base_model_recall_1.pkl', 'wb'))
    elif temp_base_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[3], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_base_model_recall_1.pkl', 'wb'))
  
  elif abs_max == temp_hyper_value: # If max value is in HYPER model.
    if temp_hyper_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[4], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_hyper_model_recall_1.pkl', 'wb'))
    elif temp_hyper_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[5], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_hyper_model_recall_1.pkl', 'wb'))
    elif temp_hyper_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[6], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_hyper_model_recall_1.pkl', 'wb'))
    elif temp_hyper_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[7], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_hyper_model_recall_1.pkl', 'wb'))

#### Accuracy 0 model pickle

In [32]:
def accuracy_0_model_pickle(accuracy_dict, model_list):
  temp_base_value = max(accuracy_dict['0']['base'].values()) # get max key, value pair of base model
  temp_base_key = list( accuracy_dict['0']['base'].keys() )[ list( accuracy_dict['0']['base'].values() ).index(temp_base_value) ]

  temp_hyper_value = max(accuracy_dict['0']['hyper'].values()) # get max key, value pair of hyper model
  temp_hyper_key = list( accuracy_dict['0']['hyper'].keys() )[ list( accuracy_dict['0']['hyper'].values() ).index(temp_hyper_value) ]

  abs_max = max(temp_base_value, temp_hyper_value) # Find which has best, base or hyper model
  
  if abs_max == temp_base_value: # If max value is in BASE model.
    if temp_base_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[0], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_base_model_accuracy_0.pkl', 'wb'))
    elif temp_base_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[1], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_base_model_accuracy_0.pkl', 'wb'))
    elif temp_base_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[2], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_base_model_accuracy_0.pkl', 'wb'))
    elif temp_base_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[3], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_base_model_accuracy_0.pkl', 'wb'))
  
  elif abs_max == temp_hyper_value: # If max value is in HYPER model.
    if temp_hyper_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[4], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_hyper_model_accuracy_0.pkl', 'wb'))
    elif temp_hyper_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[5], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_hyper_model_accuracy_0.pkl', 'wb'))
    elif temp_hyper_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[6], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_hyper_model_accuracy_0.pkl', 'wb'))
    elif temp_hyper_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[7], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_hyper_model_accuracy_0.pkl', 'wb'))

#### Precision 0 model pickle

In [33]:
def precision_0_model_pickle(precision_dict, model_list):
  temp_base_value = max(precision_dict['0']['base'].values()) # get max key, value pair of base model
  temp_base_key = list( precision_dict['0']['base'].keys() )[ list( precision_dict['0']['base'].values() ).index(temp_base_value) ]

  temp_hyper_value = max(precision_dict['0']['hyper'].values()) # get max key, value pair of hyper model
  temp_hyper_key = list( precision_dict['0']['hyper'].keys() )[ list( precision_dict['0']['hyper'].values() ).index(temp_hyper_value) ]

  abs_max = max(temp_base_value, temp_hyper_value) # Find which has best, base or hyper model
  
  if abs_max == temp_base_value: # If max value is in BASE model.
    if temp_base_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[0], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_base_model_precision_0.pkl', 'wb'))
    elif temp_base_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[1], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_base_model_precision_0.pkl', 'wb'))
    elif temp_base_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[2], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_base_model_precision_0.pkl', 'wb'))
    elif temp_base_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[3], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_base_model_precision_0.pkl', 'wb'))
  
  elif abs_max == temp_hyper_value: # If max value is in HYPER model.
    if temp_hyper_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[4], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_hyper_model_precision_0.pkl', 'wb'))
    elif temp_hyper_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[5], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_hyper_model_precision_0.pkl', 'wb'))
    elif temp_hyper_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[6], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_hyper_model_precision_0.pkl', 'wb'))
    elif temp_hyper_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[7], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_hyper_model_precision_0.pkl', 'wb'))

#### Recall 0 model pickle

In [34]:
def recall_0_model_pickle(recall_dict, model_list):
  temp_base_value = max(recall_dict['0']['base'].values()) # get max key, value pair of base model
  temp_base_key = list( recall_dict['0']['base'].keys() )[ list( recall_dict['0']['base'].values() ).index(temp_base_value) ]

  temp_hyper_value = max(recall_dict['0']['hyper'].values()) # get max key, value pair of hyper model
  temp_hyper_key = list( recall_dict['0']['hyper'].keys() )[ list( recall_dict['0']['hyper'].values() ).index(temp_hyper_value) ]

  abs_max = max(temp_base_value, temp_hyper_value) # Find which has best, base or hyper model
  
  if abs_max == temp_base_value: # If max value is in BASE model.
    if temp_base_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[0], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_base_model_recall_0.pkl', 'wb'))
    elif temp_base_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[1], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_base_model_recall_0.pkl', 'wb'))
    elif temp_base_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[2], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_base_model_recall_0.pkl', 'wb'))
    elif temp_base_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[3], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_base_model_recall_0.pkl', 'wb'))
  
  elif abs_max == temp_hyper_value: # If max value is in HYPER model.
    if temp_hyper_key == 'logReg': #If Logistic Regression is best.
      pickle.dump(model_list[4], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/logReg_hyper_model_recall_0.pkl', 'wb'))
    elif temp_hyper_key == 'SVM': # If SVM is best.
      pickle.dump(model_list[5], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/SVM_hyper_model_recall_0.pkl', 'wb'))
    elif temp_hyper_key == 'RandomForest': # If Random Forest is best.
      pickle.dump(model_list[6], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/RF_hyper_model_recall_0.pkl', 'wb'))
    elif temp_hyper_key == 'CatBoost': # If CatBoost is the best.
      pickle.dump(model_list[7], open('/content/drive/MyDrive/Great Learning/Ensemble Techniques/Ensemble Techniques Project/CAT_hyper_model_recall_0.pkl', 'wb'))

## Main Function

In [35]:
def main():
  df_preprocessed, target_col = preprocess_data() # Preprocessing done and stored in Part_B.csv file. Run this once in the end before submitting. ####################################

  #df_preprocessed = pd.read_csv('/content/drive/MyDrive/Great Learning AIML/Ensemble Techniques/Ensemble Techniques Project/Part_B.csv')
  # Use the above line for dvpt. For final submit, use first line.

  #List of Classification ML algorithms that will be used.
  '''
  1. Logistic Regression
  2. SVM
  3. Random Forest
  4. CatBoost
  '''

  cr_train_logReg_base, cr_test_logReg_base, logReg_base = LogReg_base_model(df_preprocessed.copy()) # Done and works
  cr_train_logReg_hyper, cr_test_logReg_hyper, logReg_hyper = LogReg_hyper_model(df_preprocessed.copy()) # Done and works

  cr_train_svm_base, cr_test_svm_base, svm_base = SVM_base_model(df_preprocessed.copy()) #Done and works.
  cr_train_svm_hyper, cr_test_svm_hyper, svm_hyper = SVM_hyper_model(df_preprocessed.copy()) # Done and works.

  cr_train_rf_base, cr_test_rf_base, RF_base = RF_base_model(df_preprocessed.copy()) #Done and works.
  cr_train_rf_hyper, cr_test_rf_hyper, RF_hyper = RF_hyper_model(df_preprocessed.copy()) # Done and works.

  cr_train_cat_base, cr_test_cat_base, CAT_base = CAT_base_model(df_preprocessed.copy()) #Done and works.
  cr_train_cat_hyper, cr_test_cat_hyper, CAT_hyper = CAT_hyper_model(df_preprocessed.copy()) # Done and works.

  #Let's store various metrics in dictionaries so that's it's easier to search for the best.
  accuracy_dict = {
                  '0': {
                        'base': {
                                  'logReg': cr_test_logReg_base['accuracy'],
                                  'SVM': cr_test_svm_base['accuracy'],
                                  'RandomForest': cr_test_rf_base['accuracy'],
                                  'CatBoost': cr_test_cat_base['accuracy']
                                  },
                        'hyper': {
                                'logReg': cr_test_logReg_hyper['accuracy'],
                                'SVM': cr_test_svm_hyper['accuracy'],
                                'RandomForest': cr_test_rf_hyper['accuracy'],
                                'CatBoost': cr_test_cat_hyper['accuracy']
                                }
                        },
                   '1': {
                        'base': {
                                  'logReg': cr_test_logReg_base['accuracy'],
                                  'SVM': cr_test_svm_base['accuracy'],
                                  'RandomForest': cr_test_rf_base['accuracy'],
                                  'CatBoost': cr_test_cat_base['accuracy']
                                  },
                        'hyper': {
                                'logReg': cr_test_logReg_hyper['accuracy'],
                                'SVM': cr_test_svm_hyper['accuracy'],
                                'RandomForest': cr_test_rf_hyper['accuracy'],
                                'CatBoost': cr_test_cat_hyper['accuracy']
                                }
                        }
                   }

  precision_dict = {
                    '0': {
                        'base': {
                                  'logReg': cr_test_logReg_base['0']['precision'],
                                  'SVM': cr_test_svm_base['0']['precision'],
                                  'RandomForest': cr_test_rf_base['0']['precision'],
                                  'CatBoost': cr_test_cat_base['0']['precision']
                                  },
                        'hyper': {
                                'logReg': cr_test_logReg_hyper['0']['precision'],
                                'SVM': cr_test_svm_hyper['0']['precision'],
                                'RandomForest': cr_test_rf_hyper['0']['precision'],
                                'CatBoost': cr_test_cat_hyper['0']['precision']
                                }
                        },
                   '1': {
                        'base': {
                                  'logReg': cr_test_logReg_base['1']['precision'],
                                  'SVM': cr_test_svm_base['1']['precision'],
                                  'RandomForest': cr_test_rf_base['1']['precision'],
                                  'CatBoost': cr_test_cat_base['1']['precision']
                                  },
                        'hyper': {
                                'logReg': cr_test_logReg_hyper['1']['precision'],
                                'SVM': cr_test_svm_hyper['1']['precision'],
                                'RandomForest': cr_test_rf_hyper['1']['precision'],
                                'CatBoost': cr_test_cat_hyper['1']['precision']
                                }
                        }
                    }

  recall_dict = {
                '0': {
                        'base': {
                                  'logReg': cr_test_logReg_base['0']['recall'],
                                  'SVM': cr_test_svm_base['0']['recall'],
                                  'RandomForest': cr_test_rf_base['0']['recall'],
                                  'CatBoost': cr_test_cat_base['0']['recall']
                                  },
                        'hyper': {
                                'logReg': cr_test_logReg_hyper['0']['recall'],
                                'SVM': cr_test_svm_hyper['0']['recall'],
                                'RandomForest': cr_test_rf_hyper['0']['recall'],
                                'CatBoost': cr_test_cat_hyper['0']['recall']
                                }
                        },
                   '1': {
                        'base': {
                                  'logReg': cr_test_logReg_base['1']['recall'],
                                  'SVM': cr_test_svm_base['1']['recall'],
                                  'RandomForest': cr_test_rf_base['1']['recall'],
                                  'CatBoost': cr_test_cat_base['1']['recall']
                                  },
                        'hyper': {
                                'logReg': cr_test_logReg_hyper['1']['precision'],
                                'SVM': cr_test_svm_hyper['1']['precision'],
                                'RandomForest': cr_test_rf_hyper['1']['precision'],
                                'CatBoost': cr_test_cat_hyper['1']['precision']
                                }
                        }
                }

  model_list = [logReg_base, svm_base, RF_base, CAT_base, logReg_hyper, svm_hyper, RF_hyper, CAT_hyper]

  print("Okay so done training the various models. What metric are you trying to maximize?: ")
  print("Accuracy -> 1\nPrecision -> 2\nRecall -> 3")
  decision_metric = int(input())
  print("Now which feature of target column are looking for? 1 or 0?: ", end="")
  decision_feature = input()

  if decision_feature == '1': # If looking for '1' feature

    if decision_metric == 1: # If looking for Accuracy

      accuracy_1_model_pickle(accuracy_dict, model_list)
    
    elif decision_metric == 2: # If looking for Precision.

      precision_1_model_pickle(precision_dict, model_list)
    
    elif decision_metric == 3: #If looking for Recall.

      recall_1_model_pickle(recall_dict, model_list)
  
  elif decision_feature == '0': #If looking for '0' feature

    if decision_metric == 1: #If looking for accuracy.
      
      accuracy_0_model_pickle(accuracy_dict, model_list)
    
    elif decision_metric == 2: #If looking for precision.
      
      precision_0_model_pickle(precision_dict, model_list)
    
    elif decision_metric == 3: #If looking for recall.
      
      recall_0_model_pickle(recall_dict, model_list)


In [36]:
main()

Store the csv in the following folder;
D:\Great Learning AIML\Ensemble Techniques\Ensemble Techniques Project
Type in csv file name stored in above mentioned folder: 
Part_B.csv
Enter target column name from above list: Churn
Dataframe doesn't contain any NaN values.
Dataframe doesn't contain any NaN values.
Column: gender
Unique values: ['Female' 'Male']
Assign labels to features:
Female: 0
Male: 1
Column: Partner
Unique values: ['Yes' 'No']
Assign labels to features:
Yes: 1
No: 0
Column: Dependents
Unique values: ['No' 'Yes']
Assign labels to features:
No: 0
Yes: 1
Column: PhoneService
Unique values: ['No' 'Yes']
Assign labels to features:
No: 0
Yes: 1
Column: MultipleLines
Unique values: ['No phone service' 'No' 'Yes']
['No phone service' 'No' 'Yes']
Do you want to manually encode this, Y/N?Y
Assign labels features:
No phone service: -1
No: 0
Yes: 1
Column: InternetService
Unique values: ['DSL' 'Fiber optic' 'No']
['DSL' 'Fiber optic' 'No']
Do you want to manually encode this, Y/N?N

c:\users\chirag sp\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'C': 10, 'solver': 'newton-cg'}
Logistic Regression Hypertuned model Training set
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3880
           1       0.74      0.76      0.75      3851

    accuracy                           0.75      7731
   macro avg       0.75      0.75      0.75      7731
weighted avg       0.75      0.75      0.75      7731

Logistic Regression Hypertuned model Testing set
              precision    recall  f1-score   support

           0       0.74      0.72      0.73      1294
           1       0.73      0.75      0.74      1283

    accuracy                           0.73      2577
   macro avg       0.74      0.74      0.73      2577
weighted avg       0.74      0.73      0.73      2577

SVM Base model Training set
              precision    recall  f1-score   support

           0       0.74      0.66      0.70      3880
           1       0.69      0.76      0.72      3851

    accuracy          

c:\users\chirag sp\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\users\chirag sp\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\users\chirag sp\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour,

c:\users\chirag sp\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\users\chirag sp\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\users\chirag sp\appdata\local\programs\python\python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour,

...

526:	learn: 0.2402795	total: 1.69s	remaining: 1.42s
527:	learn: 0.2401912	total: 1.7s	remaining: 1.42s
528:	learn: 0.2401594	total: 1.7s	remaining: 1.42s
529:	learn: 0.2399973	total: 1.7s	remaining: 1.41s
530:	learn: 0.2399824	total: 1.71s	remaining: 1.41s
531:	learn: 0.2398281	total: 1.71s	remaining: 1.41s
532:	learn: 0.2398108	total: 1.71s	remaining: 1.4s
533:	learn: 0.2397401	total: 1.72s	remaining: 1.4s
534:	learn: 0.2396633	total: 1.72s	remaining: 1.4s
535:	learn: 0.2396456	total: 1.72s	remaining: 1.39s
536:	learn: 0.2395671	total: 1.73s	remaining: 1.39s
537:	learn: 0.2395394	total: 1.73s	remaining: 1.39s
538:	learn: 0.2395294	total: 1.73s	remaining: 1.38s
539:	learn: 0.2394982	total: 1.73s	remaining: 1.38s
540:	learn: 0.2394477	total: 1.74s	remaining: 1.38s
541:	learn: 0.2394044	total: 1.74s	remaining: 1.37s
542:	learn: 0.2392746	total: 1.74s	remaining: 1.37s
543:	learn: 0.2391657	total: 1.75s	remaining: 1.37s
544:	learn: 0.2391509	total: 1.75s	remaining: 1.36s
545:	learn: 0.2390

690:	learn: 0.2306401	total: 2.22s	remaining: 898ms
691:	learn: 0.2306311	total: 2.23s	remaining: 895ms
692:	learn: 0.2305971	total: 2.23s	remaining: 891ms
693:	learn: 0.2305786	total: 2.23s	remaining: 888ms
694:	learn: 0.2305641	total: 2.23s	remaining: 885ms
695:	learn: 0.2305240	total: 2.24s	remaining: 881ms
696:	learn: 0.2304400	total: 2.24s	remaining: 878ms
697:	learn: 0.2304018	total: 2.25s	remaining: 875ms
698:	learn: 0.2303769	total: 2.25s	remaining: 872ms
699:	learn: 0.2303068	total: 2.25s	remaining: 869ms
700:	learn: 0.2302926	total: 2.25s	remaining: 865ms
701:	learn: 0.2302259	total: 2.26s	remaining: 862ms
702:	learn: 0.2301973	total: 2.26s	remaining: 859ms
703:	learn: 0.2301608	total: 2.26s	remaining: 856ms
704:	learn: 0.2301306	total: 2.27s	remaining: 852ms
705:	learn: 0.2300922	total: 2.27s	remaining: 849ms
706:	learn: 0.2300580	total: 2.27s	remaining: 846ms
707:	learn: 0.2300176	total: 2.28s	remaining: 843ms
708:	learn: 0.2299902	total: 2.28s	remaining: 839ms
709:	learn: 

849:	learn: 0.2245883	total: 2.77s	remaining: 391ms
850:	learn: 0.2245539	total: 2.77s	remaining: 388ms
851:	learn: 0.2245204	total: 2.78s	remaining: 385ms
852:	learn: 0.2245097	total: 2.78s	remaining: 381ms
853:	learn: 0.2244965	total: 2.78s	remaining: 378ms
854:	learn: 0.2244590	total: 2.79s	remaining: 375ms
855:	learn: 0.2244368	total: 2.79s	remaining: 372ms
856:	learn: 0.2244275	total: 2.79s	remaining: 368ms
857:	learn: 0.2244088	total: 2.79s	remaining: 365ms
858:	learn: 0.2243616	total: 2.8s	remaining: 362ms
859:	learn: 0.2243421	total: 2.8s	remaining: 358ms
860:	learn: 0.2243272	total: 2.81s	remaining: 355ms
861:	learn: 0.2242834	total: 2.81s	remaining: 352ms
862:	learn: 0.2242565	total: 2.81s	remaining: 349ms
863:	learn: 0.2242407	total: 2.81s	remaining: 345ms
864:	learn: 0.2242104	total: 2.82s	remaining: 342ms
865:	learn: 0.2240903	total: 2.82s	remaining: 339ms
866:	learn: 0.2240713	total: 2.82s	remaining: 336ms
867:	learn: 0.2240400	total: 2.83s	remaining: 332ms
868:	learn: 0.

49:	learn: 0.4503767	total: 189ms	remaining: 1.06s
50:	learn: 0.4497546	total: 193ms	remaining: 1.06s
51:	learn: 0.4484740	total: 198ms	remaining: 1.06s
52:	learn: 0.4473596	total: 202ms	remaining: 1.05s
53:	learn: 0.4464758	total: 205ms	remaining: 1.05s
54:	learn: 0.4453525	total: 210ms	remaining: 1.05s
55:	learn: 0.4446052	total: 214ms	remaining: 1.04s
56:	learn: 0.4435338	total: 217ms	remaining: 1.04s
57:	learn: 0.4431658	total: 222ms	remaining: 1.04s
58:	learn: 0.4424216	total: 226ms	remaining: 1.04s
59:	learn: 0.4416421	total: 230ms	remaining: 1.04s
60:	learn: 0.4410544	total: 235ms	remaining: 1.03s
61:	learn: 0.4407101	total: 239ms	remaining: 1.03s
62:	learn: 0.4399085	total: 243ms	remaining: 1.03s
63:	learn: 0.4392662	total: 247ms	remaining: 1.03s
64:	learn: 0.4385430	total: 251ms	remaining: 1.02s
65:	learn: 0.4379013	total: 255ms	remaining: 1.02s
66:	learn: 0.4362388	total: 260ms	remaining: 1.02s
67:	learn: 0.4354368	total: 264ms	remaining: 1.02s
68:	learn: 0.4349026	total: 268

226:	learn: 0.3578893	total: 928ms	remaining: 421ms
227:	learn: 0.3576297	total: 933ms	remaining: 417ms
228:	learn: 0.3574182	total: 937ms	remaining: 413ms
229:	learn: 0.3572891	total: 941ms	remaining: 409ms
230:	learn: 0.3570349	total: 945ms	remaining: 405ms
231:	learn: 0.3568466	total: 949ms	remaining: 401ms
232:	learn: 0.3566084	total: 953ms	remaining: 397ms
233:	learn: 0.3563031	total: 957ms	remaining: 393ms
234:	learn: 0.3560372	total: 962ms	remaining: 389ms
235:	learn: 0.3557782	total: 966ms	remaining: 385ms
236:	learn: 0.3552501	total: 970ms	remaining: 381ms
237:	learn: 0.3549642	total: 974ms	remaining: 377ms
238:	learn: 0.3546765	total: 979ms	remaining: 373ms
239:	learn: 0.3542868	total: 983ms	remaining: 369ms
240:	learn: 0.3540434	total: 987ms	remaining: 365ms
241:	learn: 0.3536734	total: 992ms	remaining: 361ms
242:	learn: 0.3533812	total: 996ms	remaining: 356ms
243:	learn: 0.3528701	total: 1000ms	remaining: 352ms
244:	learn: 0.3525608	total: 1s	remaining: 348ms
245:	learn: 0.